# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [6]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

import holoviews as hv
import cartopy
import cartopy.crs as ccrs
import geoviews as gv # noqa
import pyproj

# Import API key
from challenge6_api_keys import geoapify_key

In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head(1000)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230
...,...,...,...,...,...,...,...,...,...,...
572,572,sovetskiy,61.3614,63.5842,3.93,96,100,3.71,RU,1666108564
573,573,tatarsk,55.2190,75.9828,3.78,89,84,2.63,RU,1666108564
574,574,canchungo,12.0672,-16.0333,35.07,61,67,2.91,GW,1666108565
575,575,iquique,-20.2208,-70.1431,17.02,68,6,3.50,CL,1666108566


In [8]:
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

In [9]:
city_data_df.count()

City_ID       577
City          577
Lat           577
Lng           577
Max Temp      577
Humidity      577
Cloudiness    577
Wind Speed    577
Country       574
Date          577
dtype: int64

In [22]:
Coordinates = city_data_df[["Lat", "Lng"]]
#city_data_df["Coordinates"] = Coordinates
Coordinates

,Lat,Lng
0,18.3851,42.4509
1,58.0948,6.8047
2,-42.7826,147.0587
3,42.0970,-79.2353
4,36.0564,103.7922
...,...,...
572,61.3614,63.5842
573,55.2190,75.9828
574,12.0672,-16.0333
575,-20.2208,-70.1431


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [29]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points('Lng', 'Lat', geo=True, color='red', alpha=0.5, 
                                        xlim=(-200, 200), ylim=(-70, 70), tiles='OSM')

# Display the map plot_1
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [30]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df[(city_data_df["Max Temp"] < 29) &
                             (city_data_df["Max Temp"] > 16) &
                             (city_data_df["Wind Speed"] < 10) &
                             (city_data_df["Cloudiness"] < 15) &
                             (city_data_df["Humidity"] < 30)
                            ] 
# Drop any rows with null values
ideal_city_df=ideal_city_df.dropna()

# Display sample data
ideal_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
73,73,nalut,30.3333,10.8500,25.41,27,0,5.11,LY,1666108275
258,258,comodoro rivadavia,-45.8667,-67.5000,25.92,24,0,7.20,AR,1666108389
436,436,lakki marwat,32.6079,70.9114,27.89,20,0,1.53,PK,1666108492
550,550,pali,25.7667,73.3333,27.18,26,0,1.38,IN,1666108552


### Step 3: Create a new DataFrame called `hotel_df`.

In [31]:
coordinates_df = ideal_city_df["Lat"] + ideal_city_df["Lng"]

In [32]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
73,nalut,LY,30.3333,10.8500,27,
258,comodoro rivadavia,AR,-45.8667,-67.5000,24,
436,lakki marwat,PK,32.6079,70.9114,20,
550,pali,IN,25.7667,73.3333,26,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [41]:
# Set parameters to search for a hotel
Lat = hotel_df["Lat"]
Lng = hotel_df["Lng"]
radius = 1000000
filters = f"circle:{Lng},{Lat},{radius}"
bias = f"proximity:{Lng},{Lat}"
params = {"radius": 1000000,
          "type": "hotel",
          "key": geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = filters
    params["bias"] = bias
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
nalut - nearest hotel: No hotel found
comodoro rivadavia - nearest hotel: No hotel found
lakki marwat - nearest hotel: No hotel found
pali - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
73,nalut,LY,30.3333,10.8500,27,No hotel found
258,comodoro rivadavia,AR,-45.8667,-67.5000,24,No hotel found
436,lakki marwat,PK,32.6079,70.9114,20,No hotel found
550,pali,IN,25.7667,73.3333,26,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [39]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points('Lng', 'Lat', geo=True, color='red', alpha=0.5, 
                                        xlim=(-200, 200), ylim=(-70, 70), tiles='OSM')

# Display the map plot_1
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]